In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/drive/MyDrive/english-twi-translation-0283a70bd49c.json"


In [ ]:
!pip install google-cloud-translate
!pip install googletrans==4.0.0-rc1

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 60.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 7.0 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17397 sha256=11457ea0686eacb3c90581cb39b18e9877fa5430abf35103f2dddaa2e421d5e0
  Stored in directory: /root/.cache/pip/wheels/c0/59/9f/7372f0cf70160fe61b528532e1a7c8498c4becd6bcffb022de
Successfully built googletrans
  Attempting uninstall: chardet
    Found existing installation: chardet 4.0.0
    Uninstalling ch

In [ ]:
import pandas as pd
from google.cloud import translate_v2 as translate
from googletrans import Translator
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu


In [ ]:
# Initialize the Google Translate API
translator = translate.Client()

#  Function to translate text
def translate_text(text, target_language):
    translation = translator.translate(text, target_language=target_language)
    return translation['translatedText']

# Load the Translated test data from google api into a DataFrame
df = pd.read_csv('/content/trans_test_data_google_twi_full.csv')

df['Translated Question'] = df['Translated Question'].apply(lambda x: translate_text(x, target_language='en'))
df['Translated Desired Answer'] = df['Translated Desired Answer'].apply(lambda x: translate_text(x, target_language='en'))
df['Translated Student Answer'] = df['Translated Student Answer'].apply(lambda x: translate_text(x, target_language='en'))

translated_questions = df['Translated Question']
translated_desired_answers = df['Translated Desired Answer']
translated_student_answers = df['Translated Student Answer']

# # Display the DataFrame in a tabular form
# print(df.to_string(index=False))

translated_df = pd.DataFrame({
    'Translated Question': translated_questions,
    'Translated Desired Answer': translated_desired_answers,
    'Translated Student Answer': translated_student_answers
})

average_score = df['Average Score']
# Add the average score to the DataFrame
translated_df['Average Score'] = average_score

# Export the DataFrame to CSV
translated_df.to_csv('trans_test_data_twi_en.csv', index=False)


BLEU Score

In [ ]:
!pip install nltk

In [ ]:
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu, SmoothingFunction

# BLEU class for computing BLEU scores
class BLEU:
    def get_bleu_score(self, hypothesis, reference, smoothing):
        hypothesis_tokens = [h.strip().split() for h in hypothesis]
        reference_tokens = [[r.strip().split()] for r in reference]
        weights = (0.25, 0.25, 0.25, 0.25)  # Weight for unigrams
        smoothing_function = SmoothingFunction().method7 if smoothing else None
        bleu_score = corpus_bleu(reference_tokens, hypothesis_tokens, weights=weights, smoothing_function=smoothing_function)
        return bleu_score

# Load the test data CSV file
test_data_df = pd.read_csv("/content/test_data.csv")

# Load the translated test data CSV file
trans_test_data_df = pd.read_csv("/content/trans_test_data_twi_en.csv")

# Extract the required columns from the test data DataFrame
question_hypothesis = trans_test_data_df['Translated Question']
question_reference = test_data_df['question']

desired_answer_hypothesis = trans_test_data_df['Translated Desired Answer']
desired_answer_reference = test_data_df['desired_answer']

student_answer_hypothesis = trans_test_data_df['Translated Student Answer']
student_answer_reference = test_data_df['student_answer']

# Calculate BLEU scores for each column
bleu = BLEU()
smoothing = True  # Set to True for smoothing, False otherwise

question_bleu_score = bleu.get_bleu_score(question_hypothesis, question_reference, smoothing)
desired_answer_bleu_score = bleu.get_bleu_score(desired_answer_hypothesis, desired_answer_reference, smoothing)
student_answer_bleu_score = bleu.get_bleu_score(student_answer_hypothesis, student_answer_reference, smoothing)

# Calculate average BLEU score
average_bleu_score = (question_bleu_score + desired_answer_bleu_score + student_answer_bleu_score) / 3

# Print individual BLEU scores
print(f"Question BLEU Score: {question_bleu_score:.2f}")
print(f"Desired Answer BLEU Score: {desired_answer_bleu_score:.2f}")
print(f"Student Answer BLEU Score: {student_answer_bleu_score:.2f}")
print(f"Average BLEU Score: {average_bleu_score:.2f}")




Question BLEU Score: 0.55
Desired Answer BLEU Score: 0.51
Student Answer BLEU Score: 0.44
Average BLEU Score: 0.50


SacreBLEU Score

In [ ]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 2.8 MB/s eta 0:00:00


In [ ]:
import sacrebleu

In [ ]:
# Load the test data CSV file
test_data_df = pd.read_csv("/content/test_data.csv")

# Load the translated test data CSV file
trans_test_data_df = pd.read_csv("/content/trans_test_data_twi_en.csv")

# Extract the required columns from the test data DataFrame
question_hypothesis = list(trans_test_data_df['Translated Question'])
question_reference = [str(reference) for reference in test_data_df['question']]  # Convert single reference to a list

desired_answer_hypothesis = list(trans_test_data_df['Translated Desired Answer'])
desired_answer_reference = [str(reference) for reference in test_data_df['desired_answer']]  # Convert single reference to a list

student_answer_hypothesis = list(trans_test_data_df['Translated Student Answer'])
student_answer_reference = [str(reference) for reference in test_data_df['student_answer']]  # Convert single reference to a list

# Calculate BLEU scores for each column
question_sacrebleu_score = sacrebleu.corpus_bleu(question_hypothesis, [question_reference])
desired_answer_sacrebleu_score = sacrebleu.corpus_bleu(desired_answer_hypothesis, [desired_answer_reference])
student_answer_sacrebleu_score = sacrebleu.corpus_bleu(student_answer_hypothesis, [student_answer_reference])

# Calculate average BLEU score
average_sacrebleu_score = (
    question_sacrebleu_score.score + desired_answer_sacrebleu_score.score + student_answer_sacrebleu_score.score
) / 3

# Print individual BLEU scores
print(f"Question SacreBLEU Score:, {question_sacrebleu_score.score:.2f}")
print(f"Desired Answer SacreBLEU Score:, {desired_answer_sacrebleu_score.score:.2f}")
print(f"Student Answer SacreBLEU Score:, {student_answer_sacrebleu_score.score:.2f}")
print(f"Average SacreBLEU Score: , {average_sacrebleu_score:.2f}")


Question SacreBLEU Score:, 52.33
Desired Answer SacreBLEU Score:, 49.49
Student Answer SacreBLEU Score:, 41.49
Average SacreBLEU Score: , 47.77


Azunre BLEU

In [ ]:
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu

def calculate_bleu_score(hypothesis, reference, smoothing_function=None):
     # Tokenize sentences
    hypothesis_tokens = hypothesis.str.strip().str.lower().str.split()
    reference_tokens = reference.str.strip().str.lower().str.split()


    weights = (0.58, 0, 0, 0)  # BLEU weights
    bleu_scores = []
    for hypo, ref in zip(hypothesis_tokens, reference_tokens):
        bleu = sentence_bleu([ref], hypo, weights, smoothing_function=smoothing_function, auto_reweigh=True)
        bleu_scores.append(bleu)
    return bleu_scores

# Load the test data CSV file
test_data_csv = pd.read_csv("/content/test_data.csv")

# Load the translated test data CSV file
trans_test_data_twi_en = pd.read_csv("/content/trans_test_data_twi_en.csv")


# Select the relevant columns from the dataframes
question_hypothesis = test_data_csv['question']
question_reference = trans_test_data_twi_en['Translated Question']

desired_answer_hypothesis = test_data_csv['desired_answer']
desired_answer_reference = trans_test_data_twi_en['Translated Desired Answer']

student_answer_hypothesis = test_data_csv['student_answer']
student_answer_reference = trans_test_data_twi_en['Translated Student Answer']

# Calculate Azunre BLEU scores for each column
question_bleu_scores = calculate_bleu_score(question_hypothesis, question_reference)
desired_answer_bleu_scores = calculate_bleu_score(desired_answer_hypothesis, desired_answer_reference)
student_answer_bleu_scores = calculate_bleu_score(student_answer_hypothesis, student_answer_reference)

# Calculate the Azunre  BLEU scores
question_azunre_bleu_score = sum(question_bleu_scores) / len(question_bleu_scores)
desired_answer_azunre_bleu_score = sum(desired_answer_bleu_scores) / len(desired_answer_bleu_scores)
student_answer_azunre_bleu_score = sum(student_answer_bleu_scores) / len(student_answer_bleu_scores)
average_bleu_score = (question_azunre_bleu_score + desired_answer_azunre_bleu_score +
                      student_answer_azunre_bleu_score) / 3



/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

In [ ]:

print(f"Azunre BLEU score for Questions: {question_azunre_bleu_score:.2f}")
print(f"Azunre BLEU score for Desired Answers: {desired_answer_azunre_bleu_score:.2f}")
print(f"Azunre BLEU score for Student Answers: {student_answer_azunre_bleu_score:.2f}")
print(f"Average of Azunre BLEU score: {average_bleu_score:.2f}")



Azunre BLEU score for Questions: 0.79
Azunre BLEU score for Desired Answers: 0.67
Azunre BLEU score for Student Answers: 0.69
Average of Azunre BLEU score: 0.72
